In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load data
data = pd.read_csv("version_nueva.csv",sep=",",encoding="utf-8")

# drop some features
data = data.drop(['propertyCode','thumbnail','url','district','province',
                  'newDevelopment','title','subtitle',
                  'externalReference','priceByArea','propertyType'],axis=1)


# replace string by numbers in 'floor' column
data['floor']=data['floor'].fillna(value='0')
data = data.replace(to_replace='bj', value='0', inplace=False, limit=None, regex=False, method='pad', axis=None)
data = data.replace(to_replace='en', value='1', inplace=False, limit=None, regex=False, method='pad', axis=None)
data = data.replace(to_replace='ss', value='-1', inplace=False, limit=None, regex=False, method='pad', axis=None)
data = data.replace(to_replace='st', value='-1', inplace=False, limit=None, regex=False, method='pad', axis=None)

#Converting some selected features to float
tofloat=['price','size','rooms','floor','bathrooms','exterior']
###To be used when the floor feature is only numeric:
###tofloat=['price','priceByArea','numPhotos','size','propertyCode','rooms','floor']
for col in tofloat:
    data[col]=data[col].astype(float)

#Take only municipality Barcelona
data = data[data['municipality']=='Barcelona']
data = data.drop(['municipality'], axis=1)

################
#NEW IN VERSION 4:
# 1. Add priceperarea data from the Open data BCN external file

dataprice=pd.read_csv("LLOGUER_MITJA_MENSUAL_2016_edited.csv", encoding="utf-8")
price_columns_name=["area","priceperarea"]
dataprice.columns=price_columns_name
dataprice.loc[:,('priceperarea')] = dataprice.loc[:,('priceperarea')].map(lambda x:x.replace(',',''))
dataprice.loc[:,('priceperarea')] = pd.to_numeric(dataprice.loc[:,('priceperarea')])

left=data
right=dataprice
data=pd.merge(left, right, how='left', on=None, left_on="neighborhood", right_on="area")
data = data.drop(['area'], axis=1)

# 2. Add a combinated column about typology
data['typologycombined'] = np.where(data['subTypology'].isnull(), "type_" + data["typology"], "type_" + data["typology"] +"_"+ data["subTypology"])
data = data.drop(['typology'], axis=1)
data = data.drop(['subTypology'], axis=1)
################

data['floor'][data['floor']==1000]=0.0

def dummies_for(data, cols, binary=None):
    for col in cols:
            data = pd.concat((data, pd.get_dummies(data[col]).astype(np.int32)), axis=1)
            data = data.drop(col, axis=1)   
    return data

data = dummies_for(data,  ['typologycombined'])
data = dummies_for(data,  ['status'])
#data = dummies_for(data,  ['neighborhood'])
data = data.drop(['neighborhood'], axis=1)


data.to_csv("version4.csv",encoding="utf-8",index=False)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
